# Methodes de descente de Gradient

In [2]:
using Optim
using LinearAlgebra

f(x)=x^2+1
df(x)=2*x # admettons qu'on a la dérivée sinon on peut l'approximer avec les éléments finis

df (generic function with 1 method)

In [3]:
function bracket_minimum(f, x=0; s=1e-2, k=2.0) # Cette fonction classique permet de réduire le domaine de recherche du min
    a, ya= x,f(x)
    b, yb= a+s, f(a+s)
    if yb > ya
        a, b = b, a
        ya, yb = yb, ya
        s = -s
    end
    while true
        c, yc = b + s, f(b + s)
        if yc > yb
            return a < c ? (a, c) : (c, a)
        end
        a, ya, b, yb = b, yb, c, yc
        s *= k
    end
end  

bracket_minimum (generic function with 2 methods)

In [4]:
(a,c)=bracket_minimum(f)

(-0.01, 0.01)

In [5]:
function line_search(f,x,d) # Cette fonction sera utile pour modifier le pas
    obj=c->f(x+c*d)
    a, b = bracket_minimum(obj)
    c = Optim.optimize(obj, a, b)
    m=Optim.minimizer(c)
    return (m,x + a*d)
end


line_search (generic function with 1 method)

In [6]:
line_search(f,1,2)

(-0.5000000000000002, -1.54)

In [7]:
abstract type DescentMethod end
struct GradientDescent <: DescentMethod
    a
end
init!(M::GradientDescent, f, df, x) = M
function step(M::GradientDescent, f, df, x)
    a,g= M.a, df(x)
    return x-a*g
end

step (generic function with 1 method)

In [8]:
mutable struct GradientDescentOpt <: DescentMethod
    a
end
init!(M::GradientDescentOpt, f, df, x) = M
function step1(M::GradientDescentOpt, f, df, x)
    d=-df(x)
    M.a , x1 =line_search(f,x,d)
    return x1
end

step1 (generic function with 1 method)

In [9]:
mutable struct ConjugateGradientDescent <: DescentMethod
    d
    g
end
function init!(M::ConjugateGradientDescent, f, df, x)
    M.g = df(x)
    M.d = -M.g
    return M
end


function step2(M::ConjugateGradientDescent, f, df, x)
    d,g=M.d,M.g
    dg =df(x)
    β = max(0, dot(dg, dg-g)/(g⋅g))
    d′ = -dg + β*d
    x′ = line_search(f, x, d′)
    M.d, M.g = d′, dg
    return x′[2]
end        

step2 (generic function with 1 method)

In [10]:
M=GradientDescent(0.1)

GradientDescent(0.1)

In [11]:
i=0
s=1.
while abs(s)>0.001
   s=step(M,f,df,s)
   i=i+1
end

In [12]:
i

31

In [13]:
s

0.0009903520314283045

In [14]:
N=ConjugateGradientDescent(df(1),-df(1))

ConjugateGradientDescent(2, -2)

In [16]:
j=0
c=1.
while abs(c)>0.001
   c=step2(N,f,df,c)
   j=j+1
end

In [17]:
j

9

In [18]:
c

-0.0005232720039011942

In [19]:
L=GradientDescentOpt(0.1)

GradientDescentOpt(0.1)

In [20]:
k=0
p=1.
while abs(p)>0.0001
   p=step1(L,f,df,p)
   k=k+1
end

In [21]:
k

10

In [22]:
p

3.6561584400629754e-5